# Detection of deviation from normal development based on predicted developmental stages

# Table of Contents
* [General](#first-bullet)
* [Load paths](#second-bullet)
* [Load model](#third-bullet)
* [Calculate embeddings](#fourth-bullet)
* [(Optional) Save and load embeddings](#fifth-bullet)
* [Calculate and save similarities](#sixth-bullet)

## General <a class="anchor" id="first-bullet"></a>

General imports and class definitions

In [ ]:
%load_ext autoreload
%autoreload 2

Load paths from config file

In [ ]:
from twinnet_tools.tnconfig import ProjectConfig

config = ProjectConfig("twinnet_config")

In [ ]:
dir_root_scripts = config.json["dir_scripts"]
dir_data = config.json["dir_data"]
path_model_zebrafish = config.json["dir_model_zebrafish_original"]
paths_reference_src_images = config.json["path_json_reference_src_imgs_2"]

config_paths_script = config.json["Deviation_development_predicted_developmental_stages"]
paths_test_src_images = config_paths_script["path_src_data_test_json"]
dir_dst = config_paths_script["dir_data_dst"]

Import Python packages

In [ ]:
import glob
import matplotlib as mpl
import pandas as pd
import sys

sys.path.append(dir_root_scripts)
from twinnet_tools.tngeneral import TNToolsGeneral
from twinnet_tools.tnmodel import TNToolsNetwork
from twinnet_tools.tnarrow import TNToolsArrow
from twinnet_tools.tnarroweval import TNToolsArrowEval

from pathlib import Path

Prepare class instances

In [ ]:
tools_general = TNToolsGeneral()
tools_model = TNToolsNetwork()
tools_arrow = TNToolsArrow()
tools_arroweval = TNToolsArrowEval()

Adjust matplotlib parameters to save plots as .svg files.

In [ ]:
new_rc_params = {'text.usetex': False,
                'svg.fonttype': 'none'}
mpl.rcParams.update(new_rc_params)

## Load paths <a class="anchor" id="second-bullet"></a>

Define paths

In [ ]:
dir_dst_path = Path( dir_dst )
dir_dst_path.mkdir(parents=True, exist_ok=True)

Load reference data paths

In [ ]:
paths_reference_dst_embeddings = f"{dir_dst}/embeddings_reference"

paths_reference_dst_embeddings_path = Path( paths_reference_dst_embeddings )
paths_reference_dst_embeddings_path.mkdir(parents=True, exist_ok=True)

In [ ]:
paths_anchor = tools_general.fn_json_load(paths_reference_src_images)
paths_anchor_1 = paths_anchor["paths_anchor_1"]
paths_anchor_2 = paths_anchor["paths_anchor_2"]
paths_anchor_3 = paths_anchor["paths_anchor_3"]

# Adjust paths to directory where data is stored
paths_anchor_1 = [f"{dir_data}{p[1:]}".replace("\\", "/").replace("//", "/")
                  for p in paths_anchor_1]
paths_anchor_2 = [f"{dir_data}{p[1:]}".replace("\\", "/").replace("//", "/")
                  for p in paths_anchor_2]
paths_anchor_3 = [f"{dir_data}{p[1:]}".replace("\\", "/").replace("//", "/")
                  for p in paths_anchor_3]

Load test data paths

In [ ]:
paths_test_dst_embeddings = f"{dir_dst}/embeddings_test"

In [ ]:
dirs_embryos_test = tools_arrow.paths_embryos_from_json_fn(paths_test_src_images, dir_data)

Set output path for similarities

In [ ]:
dir_dst_similarities = f"{dir_dst}/similarities/".replace("\\", "/").replace("//", "/")

dir_dst_similarities_path = Path( dir_dst_similarities )
dir_dst_similarities_path.mkdir(parents=True, exist_ok=True)

Define parameters

In [ ]:
limit_timepoints = 250

## Load model <a class="anchor" id="third-bullet"></a>

In [ ]:
tn_model_embedding = tools_model.tn_embedding_load(path_model_zebrafish)

## Calculate embeddings <a class="anchor" id="fourth-bullet"></a>

Calculate reference embeddings

In [ ]:
embeddings_1, embeddings_2, embeddings_3 = tools_arroweval.embeddings_reference_get(
    tn_model_embedding,
    paths_anchor_1,
    paths_anchor_2,
    paths_anchor_3
)

Calculate test embeddings

In [ ]:
embeddings_test = tools_arroweval.embeddings_test_get(tn_model_embedding, dirs_embryos_test)

## Save and load embeddings <a class="anchor" id="fifth-bullet"></a>

Save reference embeddings

In [ ]:
e1_path = Path( f"{paths_reference_dst_embeddings}/embeddings_1/" )
e1_path.mkdir(parents=True, exist_ok=True)


tools_arrow.embeddings_save(embeddings_1, f"{paths_reference_dst_embeddings}/embeddings_1/")

In [ ]:
e2_path = Path( f"{paths_reference_dst_embeddings}/embeddings_2/" )
e2_path.mkdir(parents=True, exist_ok=True)

tools_arrow.embeddings_save(embeddings_2, f"{paths_reference_dst_embeddings}/embeddings_2/")

In [ ]:
e3_path = Path( f"{paths_reference_dst_embeddings}/embeddings_3/" )
e3_path.mkdir(parents=True, exist_ok=True)

tools_arrow.embeddings_save(embeddings_3, f"{paths_reference_dst_embeddings}/embeddings_3/")

Load reference embeddings

In [ ]:
embeddings_1 = tools_arrow.embeddings_load(f"{paths_reference_dst_embeddings}/embeddings_1/")

In [ ]:
embeddings_2 = tools_arrow.embeddings_load(f"{paths_reference_dst_embeddings}/embeddings_2/")

In [ ]:
embeddings_3 = tools_arrow.embeddings_load(f"{paths_reference_dst_embeddings}/embeddings_3/")

Save test embeddings

In [ ]:
tools_arrow.embeddings_test_save(embeddings_test, paths_test_dst_embeddings)

Load test embeddings

In [ ]:
embeddings_test = tools_arrow.embeddings_test_load(paths_test_dst_embeddings)

## Calculate and save similarities <a class="anchor" id="sixth-bullet"></a>

In [ ]:
tools_arrow.similarities_calc(embeddings_1,
                              embeddings_2,
                              embeddings_3,
                              embeddings_test,
                              dir_dst_similarities,
                              limit_indices=250)